In [1]:
import pandas as pd
import numpy as np

import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()


22/05/19 15:06:53 WARN Utils: Your hostname, Abdiels-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.96 instead (on interface en0)
22/05/19 15:06:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/19 15:06:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


1. Create a spark data frame that contains your favorite programming languages.

    - The name of the column should be language
    - View the schema of the dataframe
    - Output the shape of the dataframe
    - Show the first 5 records in the dataframe

In [2]:
# Create 
pandas_df = pd.DataFrame({'language':['python', 'mysql', 'html', 'c++', 'css', 'java']})

df = spark.createDataFrame(pandas_df)

In [3]:
# View the Schema of the dataframe
df.printSchema()

root
 |-- language: string (nullable = true)



In [4]:
# output the shape
print((df.count(), len(df.columns)))

(6, 1)


In [5]:
# show the first 5 records 
df.show(5)

+--------+
|language|
+--------+
|  python|
|   mysql|
|    html|
|     c++|
|     css|
+--------+
only showing top 5 rows



2. Load the mpg dataset as a spark dataframe.

    - a. Create 1 column of output that contains a message like the one below:
        > The 1999 audi a4 has a 4 cylinder engine.
    
        For each vehicle.

    - b. Transform the trans column so that it only contains either manual or auto.

In [6]:
# import data function
from pydataset import data
# assign the data to a variable
mpg = spark.createDataFrame(data('mpg'))
# ensure it worked
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [7]:
# import pyspark functions
from pyspark.sql.functions import concat, lit, sum, avg, min, max, count, mean, when

# Create a column that contains message like listed above for each vehicle
mpg.select(
    concat(lit("The "), 
    mpg.year,lit(' '), 
    mpg.manufacturer, lit(' '),  
    mpg.model, lit(' has a '), 
    mpg.cyl, lit(' cylinder engine.')
    ).alias('Description')
).show(truncate=False)


+--------------------------------------------------------------+
|Description                                                   |
+--------------------------------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 2008 audi a4 has a 4 cylinder engine.                     |
|The 2008 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 2008 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 2008 audi a4 quattro has a 4 cylinder engine.             |
|The 2008 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 quattro has a 6 cylinder engine.             |
|The 1999 audi a4 quattro

In [8]:
# Import when
from pyspark.sql.functions import when

# Transform the trans column so that it only contains either manual or auto
mpg.select(
    when(
        mpg.trans.like('manual%'), 'manual')
        .otherwise('automatic')
        .alias('Transmision')
    ).show(truncate=False)


+-----------+
|Transmision|
+-----------+
|automatic  |
|manual     |
|manual     |
|automatic  |
|automatic  |
|manual     |
|automatic  |
|manual     |
|automatic  |
|manual     |
|automatic  |
|automatic  |
|manual     |
|automatic  |
|manual     |
|automatic  |
|automatic  |
|automatic  |
|automatic  |
|automatic  |
+-----------+
only showing top 20 rows



3. Load the tips dataset as a spark dataframe.

    - a. What percentage of observations are smokers?
    - b. Create a column that contains the tip percentage
    - c. Calculate the average tip percentage for each combination of sex and smoker.

In [9]:
# Load the tips dataset as a spark dataframe.
tips = spark.createDataFrame(data('tips'))
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [56]:
# Import col and round
from pyspark.sql.functions import col, round

# What percentage of observations are smokers? 
# ====> 38.11%
tips.groupBy('smoker').count().withColumn('percent', 
    (round((col('count')/tips.count()*100), 2))
).show()


+------+-----+-------+
|smoker|count|percent|
+------+-----+-------+
|    No|  151|  61.89|
|   Yes|   93|  38.11|
+------+-----+-------+



In [24]:
# Create a column that contains the tip percentage
tips.withColumn("tip_percentage (%)", round(col("tip") / col("total_bill")*100, 2)).show()

+----------+----+------+------+---+------+----+------------------+
|total_bill| tip|   sex|smoker|day|  time|size|tip_percentage (%)|
+----------+----+------+------+---+------+----+------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|              5.94|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|             16.05|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|             16.66|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|             13.98|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|             14.68|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|             18.62|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|             22.81|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|             11.61|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|             13.03|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|             21.85|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|             16.65|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|             14

In [55]:
# Calculate the average tip percentage for each combination of sex and smoker.
(
    tips.withColumn("tip_percentage", round(col("tip") / col("total_bill")*100,2))
    .groupby("sex")
    .pivot("smoker")
    .agg(round(mean("tip_percentage"), 2))
    .show()
)

+------+-----+-----+
|   sex|   No|  Yes|
+------+-----+-----+
|Female|15.69|18.21|
|  Male|16.07|15.28|
+------+-----+-----+



4. Use the seattle weather dataset referenced in the lesson to answer the questions below.

    - Convert the temperatures to fahrenheit.
    - Which month has the most rain, on average?
    - Which year was the windiest?
    - What is the most frequent type of weather in January?
    - What is the average high and low temperature on sunny days in July in 2013 and 2014?
    - What percentage of days were rainy in q3 of 2015?
    - For each year, find what percentage of days it rained (had non-zero precipitation).

In [30]:
# import data 
from vega_datasets import data

weather = spark.createDataFrame(data.seattle_weather())
weather.show(4)

+-------------------+-------------+--------+--------+----+-------+
|               date|precipitation|temp_max|temp_min|wind|weather|
+-------------------+-------------+--------+--------+----+-------+
|2012-01-01 00:00:00|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02 00:00:00|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03 00:00:00|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04 00:00:00|         20.3|    12.2|     5.6| 4.7|   rain|
+-------------------+-------------+--------+--------+----+-------+
only showing top 4 rows



In [34]:
# Convert the temperatures to fahrenheit
# F = C * 1.8 + 32
weather.withColumn(
    "temp_max", round((col("temp_max") * 1.8 + 32),2)
).withColumn("temp_min", round((col("temp_min") * 1.8 + 32), 2)).show()

+-------------------+-------------+--------+--------+----+-------+
|               date|precipitation|temp_max|temp_min|wind|weather|
+-------------------+-------------+--------+--------+----+-------+
|2012-01-01 00:00:00|          0.0|   55.04|    41.0| 4.7|drizzle|
|2012-01-02 00:00:00|         10.9|   51.08|   37.04| 4.5|   rain|
|2012-01-03 00:00:00|          0.8|   53.06|   44.96| 2.3|   rain|
|2012-01-04 00:00:00|         20.3|   53.96|   42.08| 4.7|   rain|
|2012-01-05 00:00:00|          1.3|   48.02|   37.04| 6.1|   rain|
|2012-01-06 00:00:00|          2.5|   39.92|   35.96| 2.2|   rain|
|2012-01-07 00:00:00|          0.0|   44.96|   37.04| 2.3|   rain|
|2012-01-08 00:00:00|          0.0|    50.0|   37.04| 2.0|    sun|
|2012-01-09 00:00:00|          4.3|   48.92|    41.0| 3.4|   rain|
|2012-01-10 00:00:00|          1.0|   42.98|   33.08| 3.4|   rain|
|2012-01-11 00:00:00|          0.0|   42.98|   30.02| 5.1|    sun|
|2012-01-12 00:00:00|          0.0|   42.98|   28.94| 1.9|    

In [41]:
# add pyspark month, year, and quarter function
from pyspark.sql.functions import month, year, quarter

# Which month has the most rain, on average?
(
    weather.withColumn("month", month("date"))
    .withColumn("year", year("date"))
    .groupBy("month", "year")
    .agg(sum("precipitation").alias("total_monthly_precipitation"))
    .groupBy("month")
    .agg(mean("total_monthly_precipitation").alias("avg_monthly_rain"))
    .sort(col("avg_monthly_rain").desc())
    .first()
)

Row(month=11, avg_monthly_rain=160.625)

In [49]:
# Which year was the windiest?
(
    weather.withColumn("year", year("date"))
    .groupBy("year")
    .agg(round(mean("wind"),2).alias("total_winds"))
    .sort(col("total_winds").desc())
    .first()
)

Row(year=2012, total_winds=3.4)

In [52]:
# What is the most frequent type of weather in January?
(
    weather.withColumn("month", month("date"))
    .filter(col("month") == 1)
    .groupBy("weather")
    .count()
    .sort(col("count").desc())
    .first()
)


Row(weather='fog', count=38)

In [69]:
# What is the average high and low temperature on sunny days in July in 2013 and 2014?
(
    weather.filter(month("date") == 7)
    .filter(year("date") > 2012)
    .filter(year("date") < 2015)
    .filter(col("weather") == lit("sun"))
    .agg(
        round(avg("temp_max"),2).alias("average_high_temp"),
        round(avg("temp_min"),2).alias("average_low_temp"),
    )
    .show()
)

+-----------------+----------------+
|average_high_temp|average_low_temp|
+-----------------+----------------+
|            26.83|           14.18|
+-----------------+----------------+



In [66]:
# What percentage of days were rainy in q3 of 2015?
(
    weather.filter(year("date") == 2015)
    .filter(quarter("date") == 3)
    .select(when(col("weather") == "rain", 1).otherwise(0).alias("rain"))
    .agg(round(mean("rain"),4).alias('avg rain'))
    .show()
)

+--------+
|avg rain|
+--------+
|  0.0217|
+--------+



In [65]:
# For each year, find what percentage of days it rained (had non-zero precipitation)
(
    weather.withColumn("year", year("date"))
    .select(when(col("precipitation") > 0, 1).otherwise(0).alias("did_rain"), "year")
    .groupby("year")
    .agg(round(mean("did_rain"), 4).alias('avg rain'))
    .show()
)

+----+--------+
|year|avg rain|
+----+--------+
|2012|  0.4836|
|2013|  0.4164|
|2014|   0.411|
|2015|  0.3945|
+----+--------+

